In [1]:

import pandas as pd
import numpy as np

In [2]:
movies_df = pd.read_csv(r'c:\Users\AVINASH RABHADIA\Desktop\Datasets\KNNmovie\movies.csv',usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})


In [3]:
rating_df=pd.read_csv(r'c:\Users\AVINASH RABHADIA\Desktop\Datasets\KNNmovie\mratings.csv',usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [52]:
movies_df.head()

movieId                               title
0        1                    Toy Story (1995)
1        2                      Jumanji (1995)
2        3             Grumpier Old Men (1995)
3        4            Waiting to Exhale (1995)
4        5  Father of the Bride Part II (1995)

In [53]:
rating_df.head()

userId  movieId  rating
0       1        1   4.000
1       1        3   4.000
2       1        6   4.000
3       1       47   5.000
4       1       50   5.000

In [4]:
df = pd.merge(rating_df,movies_df,on='movieId')
df.head()

userId  movieId  rating             title
0       1        1     4.0  Toy Story (1995)
1       5        1     4.0  Toy Story (1995)
2       7        1     4.5  Toy Story (1995)
3      15        1     2.5  Toy Story (1995)
4      17        1     4.5  Toy Story (1995)

In [5]:
combine_movie_rating = df.dropna(axis = 0, subset = ['title'])

In [14]:
movie_ratingCount = combine_movie_rating.groupby(by = ['title'])['rating']

In [15]:
movie_ratingCount.head()

0         4.0
1         4.0
2         4.5
3         2.5
4         4.5
         ... 
100831    2.5
100832    4.5
100833    3.0
100834    3.5
100835    3.5
Name: rating, Length: 28808, dtype: float32

In [16]:
movie_ratingCount.count()

title
'71 (2014)                                    1
'Hellboy': The Seeds of Creation (2004)       1
'Round Midnight (1986)                        2
'Salem's Lot (2004)                           1
'Til There Was You (1997)                     2
                                             ..
eXistenZ (1999)                              22
xXx (2002)                                   24
xXx: State of the Union (2005)                5
¡Three Amigos! (1986)                        26
À nous la liberté (Freedom for Us) (1931)     1
Name: rating, Length: 9719, dtype: int64

In [19]:
movie_ratingCount = movie_ratingCount.count().reset_index()

In [20]:
movie_ratingCount

title  rating
0                                    '71 (2014)       1
1       'Hellboy': The Seeds of Creation (2004)       1
2                        'Round Midnight (1986)       2
3                           'Salem's Lot (2004)       1
4                     'Til There Was You (1997)       2
...                                         ...     ...
9714                            eXistenZ (1999)      22
9715                                 xXx (2002)      24
9716             xXx: State of the Union (2005)       5
9717                      ¡Three Amigos! (1986)      26
9718  À nous la liberté (Freedom for Us) (1931)       1

[9719 rows x 2 columns]

In [21]:
movie_ratingCount = movie_ratingCount.rename(columns = {'rating': 'totalRatingCount'})

In [22]:
movie_ratingCount

title  totalRatingCount
0                                    '71 (2014)                 1
1       'Hellboy': The Seeds of Creation (2004)                 1
2                        'Round Midnight (1986)                 2
3                           'Salem's Lot (2004)                 1
4                     'Til There Was You (1997)                 2
...                                         ...               ...
9714                            eXistenZ (1999)                22
9715                                 xXx (2002)                24
9716             xXx: State of the Union (2005)                 5
9717                      ¡Three Amigos! (1986)                26
9718  À nous la liberté (Freedom for Us) (1931)                 1

[9719 rows x 2 columns]

In [23]:
movie_ratingCount.head()

title  totalRatingCount
0                               '71 (2014)                 1
1  'Hellboy': The Seeds of Creation (2004)                 1
2                   'Round Midnight (1986)                 2
3                      'Salem's Lot (2004)                 1
4                'Til There Was You (1997)                 2

In [24]:
combine_movie_rating.head()

userId  movieId  rating             title
0       1        1     4.0  Toy Story (1995)
1       5        1     4.0  Toy Story (1995)
2       7        1     4.5  Toy Story (1995)
3      15        1     2.5  Toy Story (1995)
4      17        1     4.5  Toy Story (1995)

In [25]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')

In [26]:
rating_with_totalRatingCount.head()

userId  movieId  rating             title  totalRatingCount
0       1        1     4.0  Toy Story (1995)               215
1       5        1     4.0  Toy Story (1995)               215
2       7        1     4.5  Toy Story (1995)               215
3      15        1     2.5  Toy Story (1995)               215
4      17        1     4.5  Toy Story (1995)               215

In [27]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [28]:
print(movie_ratingCount['totalRatingCount'].describe())

count   9719.000
mean      10.375
std       22.406
min        1.000
25%        1.000
50%        3.000
75%        9.000
max      329.000
Name: totalRatingCount, dtype: float64


In [29]:
popularity_threshold = 50

In [30]:
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')

In [31]:
rating_popular_movie.head()

userId  movieId  rating             title  totalRatingCount
0       1        1   4.000  Toy Story (1995)               215
1       5        1   4.000  Toy Story (1995)               215
2       7        1   4.500  Toy Story (1995)               215
3      15        1   2.500  Toy Story (1995)               215
4      17        1   4.500  Toy Story (1995)               215

In [33]:
len(rating_with_totalRatingCount)

100836

In [32]:
len(rating_popular_movie)

41362

In [34]:
movie_features_df = rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df.head()

userId                              1     2     3     4     5     6     7    \
title                                                                         
10 Things I Hate About You (1999) 0.000 0.000 0.000 0.000 0.000 0.000 0.000   
12 Angry Men (1957)               0.000 0.000 0.000 5.000 0.000 0.000 0.000   
2001: A Space Odyssey (1968)      0.000 0.000 0.000 0.000 0.000 0.000 4.000   
28 Days Later (2002)              0.000 0.000 0.000 0.000 0.000 0.000 0.000   
300 (2007)                        0.000 0.000 0.000 0.000 0.000 0.000 0.000   

userId                              8     9     10   ...   601   602   603  \
title                                                ...                     
10 Things I Hate About You (1999) 0.000 0.000 0.000  ... 0.000 0.000 3.000   
12 Angry Men (1957)               0.000 0.000 0.000  ... 5.000 0.000 0.000   
2001: A Space Odyssey (1968)      0.000 0.000 0.000  ... 0.000 0.000 5.000   
28 Days Later (2002)              0.000 0.000 0.000  ... 0.000 0.000 0.000   
300 (2007)                        0.000 0.000 3.000  ... 0.000 0.000 0.000   

userId                              604   605   606   607   608   609   610  
title                                                                        
10 Things I Hate About You (1999) 0.000 5.000 0.000 0.000 0.000 0.000 0.000  
12 Angry Men (1957)               0.000 0.000 0.000 0.000 0.000 0.000 0.000  
2001: A Space Odyssey (1968)      0.000 0.000 5.000 0.000 3.000 0.000 4.500  
28 Days Later (2002)              0.000 0.000 0.000 0.000 3.500 0.000 5.000  
300 (2007)                        0.000 3.000 0.000 0.000 5.000 0.000 4.000  

[5 rows x 606 columns]

In [35]:
from scipy.sparse import csr_matrix

In [36]:
movie_features_df_matrix = csr_matrix(movie_features_df.values)

In [37]:
movie_features_df_matrix

<450x606 sparse matrix of type '<class 'numpy.float32'>'
	with 41360 stored elements in Compressed Sparse Row format>

In [38]:
from sklearn.neighbors import NearestNeighbors

In [39]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

In [40]:
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [41]:
movie_features_df.shape

(450, 606)

In [44]:
movie_features_df.shape[0]

450

In [42]:
query_index = np.random.choice(movie_features_df.shape[0])

In [43]:
print(query_index)

419


In [45]:
distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

In [46]:
distances

array([[0.        , 0.43230182, 0.4377867 , 0.44044697, 0.45821017,
        0.46566182]], dtype=float32)

In [47]:
indices

array([[419, 318, 394, 214, 346, 149]], dtype=int64)

In [51]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {}:\n'.format(movie_features_df.index[query_index]))
    else:
        print('{}: {}, with distance of {}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Twelve Monkeys (a.k.a. 12 Monkeys) (1995):

1: Pulp Fiction (1994), with distance of 0.4323018193244934:
2: Terminator 2: Judgment Day (1991), with distance of 0.43778669834136963:
3: Independence Day (a.k.a. ID4) (1996), with distance of 0.44044697284698486:
4: Seven (a.k.a. Se7en) (1995), with distance of 0.45821017026901245:
5: Fargo (1996), with distance of 0.46566182374954224:
